In [1]:
#default_exp word_embeddings

In [2]:
#export

import os
import sys
import tqdm

from sklearn import feature_extraction, metrics
import numpy as np
import pandas as pd

import mlutil.parallel

import nltk
from gensim.models import doc2vec

import gensim
from github_search import repository_descriptions

In [3]:
%cd ..

/home/kuba/Projects/github_search


In [4]:
all_codesearch_df = repository_descriptions.get_all_codesearch_df('data/python')
repos_descriptions_df = repository_descriptions.load_pypi_repo_descriptions()

In [5]:
codesearch_df = all_codesearch_df[all_codesearch_df['repo'].isin(repos_descriptions_df['repo'])]

In [6]:
train_codesearch_df = codesearch_df[codesearch_df['partition'] == 'train']

In [7]:
train_comments = train_codesearch_df['docstring']

In [8]:
repo_idx = train_codesearch_df['repo'].astype('category').cat.codes
train_codesearch_df['repo_idx'] = repo_idx

<ipython-input-8-49067e2b050c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_codesearch_df['repo_idx'] = repo_idx


In [9]:
train_descriptions = repos_descriptions_df.merge(train_codesearch_df[['repo', 'repo_idx']], on='repo').drop_duplicates('repo')

In [10]:
train_descriptions.shape

(8036, 3)

In [11]:
def make_document(words, doc_label):
    return doc2vec.TaggedDocument(nltk.word_tokenize(words.lower()), [doc_label])

In [12]:
description_documents = train_descriptions.apply(lambda x: make_document(x.pypi_description, x.repo_idx), axis=1)

In [13]:
docstring_documents = train_codesearch_df.apply(lambda x: make_document(x.docstring, x.repo_idx), axis=1)

In [14]:
corpus = pd.concat([docstring_documents, description_documents])

In [15]:
corpus.iloc[0]

TaggedDocument(words=['return', 'either', 'the', 'full', 'or', 'truncated', 'version', 'of', 'a', 'qiime-formatted', 'taxonomy', 'string', '.', ':', 'type', 'p', ':', 'str', ':', 'param', 'p', ':', 'a', 'qiime-formatted', 'taxonomy', 'string', ':', 'k__foo', ';', 'p__bar', ';', '...', ':', 'type', 'level', ':', 'str', ':', 'param', 'level', ':', 'the', 'different', 'level', 'of', 'identification', 'are', 'kingdom', '(', 'k', ')', ',', 'phylum', '(', 'p', ')', ',', 'class', '(', 'c', ')', ',', 'order', '(', 'o', ')', ',', 'family', '(', 'f', ')', ',', 'genus', '(', 'g', ')', 'and', 'species', '(', 's', ')', '.', 'if', 'level', 'is', 'not', 'provided', ',', 'the', 'default', 'level', 'of', 'identification', 'is', 'species', '.', ':', 'rtype', ':', 'str', ':', 'return', ':', 'a', 'qiime-formatted', 'taxonomy', 'string', 'up', 'to', 'the', 'classification', 'given', 'by', 'param', 'level', '.'], tags=[6911])

In [16]:
%%time
doc2vec_model = gensim.models.Doc2Vec(list(corpus))

CPU times: user 1min 37s, sys: 5.67 s, total: 1min 42s
Wall time: 59.6 s


In [17]:
doc2vec_model.infer_vector(description_documents[0].words)

array([ 1.1451368e-01, -3.1799287e-02, -6.0764186e-02,  4.1855898e-02,
       -7.0309997e-02, -1.2279561e-01,  4.0411081e-02, -4.3489143e-02,
        5.9532339e-04,  8.9765135e-03,  6.9588654e-02,  8.7416828e-02,
        3.4857027e-02, -9.5076010e-02,  1.1034787e-01, -2.5430758e-02,
        1.1329495e-01, -8.9463685e-03,  1.9726587e-02, -1.5841305e-02,
        6.6286154e-02, -2.0812349e-02,  2.6917471e-02,  5.5228774e-02,
        2.2524416e-02, -9.4501048e-02, -3.3865531e-03, -4.3131191e-02,
       -2.3321824e-03, -2.7688961e-02, -1.6715537e-03, -4.7803126e-02,
        4.1988578e-02, -5.3104825e-02,  2.2439675e-02, -3.3608079e-02,
        6.6184476e-02, -1.2888160e-02,  6.4826079e-02,  6.8501100e-02,
        7.2375208e-02,  5.0392300e-02,  9.9858068e-02,  1.4047024e-02,
       -1.0003498e-01,  5.4971159e-02,  1.4228437e-02, -7.5144200e-03,
       -5.0331317e-02, -4.6649739e-02, -3.4678720e-02,  2.4093682e-02,
        2.7728709e-03, -4.2118549e-02, -4.9761375e-03,  1.6864989e-04,
      

In [18]:
description_documents.iloc[1].words

['open-source',
 'algorithms',
 'for',
 'data-driven',
 'building',
 'analysis',
 'and',
 'control']